# Bernoulli KNN

In [6]:
from sigmaepsilon import BeamSection, get_section

E = 2100.0  # Young's modulus
nu = 0.3  # Poisson's ratio

section = get_section('CHS', D=1.0, t=0.2, n=64)
beam = BeamSection(wrap=section)

In [7]:
beam.calculate_geometric_properties()
props = beam.geometric_properties
A = props['A']
Iy = props['Iy']
Iz = props['Iz']
Ix = props['Ix']

In [8]:
from sigmaepsilon import Structure, LineMesh, PointData
from sigmaepsilon.math.linalg import linspace, Vector
from sigmaepsilon.mesh.space import StandardFrame, PointCloud, frames_of_lines
from sigmaepsilon.fem.cells import B2 as Beam
from dewloosh.mesh.grid import knngridL2
import numpy as np

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

Lx, Ly, Lz = 20., 12., 8.

# mesh
gridparams = {
    'size': (Lx, Ly, Lz),
    'shape': (20, 13, 10),
    'eshape': (2, 2, 2),
    'origo': (0, 0, 0),
    'start': 0
}
coords, topo = knngridL2(k=10, **gridparams)
points = PointCloud(coords, frame=GlobalFrame).centralize()
dx = - np.array([points[:, 0].min(), 0., 0.])
points.move(dx)
coords = points.show()

# essential boundary conditions
ebcinds = np.where(coords[:, 0] < 1e-12)[0]
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[ebcinds, :] = True

# natural boundary conditions
loads = np.zeros((coords.shape[0], 6))
pA = points.closest(np.array([Lx, 0., 0.]))
loads[pA.gid, 2] = -1000

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)

cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)



In [9]:
structure.preprocess()
gnum = structure._gnum
K_bulk_0 = np.copy(structure._K_bulk)
vols = structure.volumes()
centers = structure.centers()

AttributeError: 'Structure' object has no attribute                 called _gnum

In [ ]:
structure.linsolve(summary=True)
structure.summary

In [4]:
from pyvistaqt import BackgroundPlotter

mesh.config['pyvista', 'plot', 'line_width'] = 1

plotter = BackgroundPlotter()
mesh.pvplot(plotter=plotter)

plotter.show()

In [5]:
#structure.linsolve(summary=True)
dofsol = structure.nodal_dof_solution()[:, :3]
dofsol[:, 2].min()

-1.8989112324550048